# Compare samplers

In this notebook, we'll compare the different samplers implemented in `bilby`. As of this version, we don't compare the outputs, only how to run them and the timings for their default setup.

## Setup

In [ ]:
import numpy as np
import pylab as plt

%load_ext autoreload
%autoreload 2

import bilby

bilby.utils.setup_logger()

time_duration = 1.
sampling_frequency = 4096.

injection_parameters = dict(mass_1=36., mass_2=29., a_1=0, a_2=0, tilt_1=0, tilt_2=0, phi_12=0, phi_jl=0,
                            luminosity_distance=100., iota=0.4, phase=1.3, waveform_approximant='IMRPhenomPv2',
                            reference_frequency=50., ra=1.375, dec=-1.2108, geocent_time=1126259642.413,
                            psi=2.659)


waveform_generator = bilby.gw.waveform_generator.WaveformGenerator(
    sampling_frequency=sampling_frequency,
    time_duration=time_duration,
    frequency_domain_source_model=bilby.gw.source.lal_binary_black_hole,
    parameters=injection_parameters)

hf_signal = waveform_generator.frequency_domain_strain()

H1 = bilby.gw.detector.get_empty_interferometer('H1')
H1.set_data(sampling_frequency=sampling_frequency, duration=time_duration,
            from_power_spectral_density=True)
H1.inject_signal(waveform_polarizations=hf_signal, parameters=injection_parameters)

IFOs = [H1]

likelihood = bilby.gw.likelihood.GravitationalWaveTransient(IFOs, waveform_generator)

## Prior

For this test, we will simply search of the sky position, setting the other parameters to their simulated values.

In [ ]:
priors = bilby.core.prior.fill_priors(injection_parameters, likelihood)
priors['ra'] = bilby.prior.Uniform(0, 2*np.pi, 'ra')
priors['dec'] = bilby.prior.Uniform(-np.pi/2, np.pi/2, 'dec')

## PyMultinest

In [ ]:
%%time 
result = bilby.core.sampler.run_sampler(
    likelihood, priors=priors, sampler='pymultinest', label='pymultinest',
    npoints=200, verbose=False, resume=False)
fig = result.plot_corner(save=False)
plt.show()
print(result)

# dynesty

In [ ]:
%%time 
result = bilby.gw.sampler.run_sampler(
    likelihood, priors=priors, sampler='dynesty', label='dynesty',
    bound='multi', sample='rwalk', npoints=200, walks=1, verbose=False,
    update_interval=100)
fig = result.plot_corner(save=False)
plt.show()
print(result)

# Dynamic Nested Sampling (Dynesty)

See [the dynesty docs](http://dynesty.readthedocs.io/en/latest/dynamic.html#). Essentially, this methods improves the posterior estimation over that of standard nested sampling.

In [ ]:
%%time 
result = bilby.gw.sampler.run_sampler(
    likelihood, priors=priors, sampler='dynesty', label='dynesty_dynamic',
    bound='multi', nlive=250, sample='unif', verbose=True,
    update_interval=100, dynamic=True)
fig = result.plot_corner(save=False)
plt.show()
print(result)

# ptemcee

In [ ]:
%%time 
result = bilby.gw.sampler.run_sampler(
    likelihood, priors=priors, sampler='ptemcee', label='ptemcee',
    nwalkers=100, nsteps=200, nburn=100, ntemps=2,
    tqdm='tqdm_notebook')
fig = result.plot_corner(save=False)
plt.show()
print(result)